using Julia Turing.jl

In [ ]:
using Turing, Distributions, DifferentialEquations, DiffEqBayes, DelimitedFiles
using MCMCChains, Plots, StatsPlots
using DataFrames, CSV

# Seed
using Random
Random.seed!(96)

# Logs
using Logging
Logging.disable_logging(Logging.Warn)
LogLevel(1001);

#### Run settings

In [ ]:
# Folders for these runs
figSaveFolder = "Figures"
paramSaveFolder = "Param Fits"

# Create the folders if they don't exist
mkpath("ODE Model/"*figSaveFolder)
mkpath("ODE Model/"*paramSaveFolder);

In [ ]:
# Dataset strings
dataset_strings = ["1-Pastushenko", "2-vanDijk", "3-Cook"]

# Condition strings
pastushenko_conditions = ["Pastushenko"]

# File strings
pseudotime_str = " - Pseudotime - 15 bins, cropped.csv"
stdev_str = " - Pseudotime - 15 bins, cropped, stdev.csv"

vanDijk_files = []
vanDijk_files_stdev = []
for currFile in vanDijk_conditions
    push!(vanDijk_files, currFile*pseudotime_str)
    push!(vanDijk_files_stdev, currFile*stdev_str)
end

cook_files_pseudotime = []
cook_files_stdev = []
for currFile in cook_conditions
    push!(cook_files_pseudotime, currFile*pseudotime_str)
    push!(cook_files_stdev, currFile*stdev_str)
end

## Signaling_Model_Functions

### ODE Model

In [ ]:
function model_equations(dCell_types, cell_types, model_params, model_time)
    E, I, M = cell_types
    k1, k2  = model_params
    dCell_types[1] = dE = -k1*E*I
    dCell_types[2] = dI = k1*E*I - k2*I*M
    dCell_types[3] = dM = k2*I*M
end;

unfitted_model \
calculate and graph model with unfitted, arbitrary parameters

In [ ]:
init_cell_proportions = [0.998, 0.001, 0.001]
legend_labels = ["E" "I" "M"]
line_colors = ["#D95319" "#EDB120" "#0072BD"]

function unfitted_model()
    
    # Solve ODE with unfitted parameters
    
    unfitted_kinetic_param = 1.0
    unfitted_params = [unfitted_kinetic_param, unfitted_kinetic_param]

    global init_cell_proportions = [0.998, 0.001, 0.001]
    time = (0.0, 25.0)

    ode_prob_unfitted = ODEProblem(model_equations, init_cell_proportions, time, unfitted_params)
    ode_sol_unfitted = solve(ode_prob_unfitted, Tsit5())
    
    # Graph unfitted solution
    
    legend_labels = ["E" "I" "M"]
    line_colors = ["#D95319" "#EDB120" "#0072BD"]

    plot(ode_sol_unfitted, lw=3, lc=line_colors, legend=:right, labels=legend_labels, dpi=250)
    title!("Signaling Model\nUnfitted Parameters")
    xlabel!("Pseudotime")
    ylabel!("Proportion of Cells in Population")
    ylims!(0.0,1.0)
    
    # Save figure
    
    cd("ODE Model/")
    savefig("Signaling Model w Unfitted Parameters.png")
    
    return # return nothing
    println(init_cell_proportions)
end;

### Data Fitting

In [ ]:
# code inspiration: https://turing.ml/dev/tutorials/10-bayesian-differential-equations/

@model function fit_model(data, data_timepoints)
    
    # Noise param distribution
    σ ~ InverseGamma(3, 1)

    # Kinetic param distributions
    # General note: pretty uninformative priors
    # Truncate values <= 0 because kinetic parameters can't be negative
    k1_mean = 4.0
    k2_mean = 4.0
    k_stdev = 1.0
    truncated_lower = 0.001
    truncated_upper = 10.0
    k1 ~ truncated(Normal(k1_mean,k_stdev), truncated_lower,truncated_upper)
    k2 ~ truncated(Normal(k2_mean,k_stdev), truncated_lower,truncated_upper)
    model_params = [k1,k2]
    
    # Solve ODE using priors
    ode_prob_priors = ODEProblem(model_equations, init_cell_proportions, (0.0,10.0), model_params)
    ode_sol_priors = solve(ode_prob_priors,Tsit5(),saveat=data_timepoints[2]) # Save at increments that match data_timepoint increments
    
    # Using predicted solve values, set priors for data per timepoint
    for i = 1:size(data,2) # Each column (timepoint)
        data[:,i] ~ MvNormal(ode_sol_priors[i], σ)
    end
    
end;

In [ ]:
function create_data_model(datasetStr, fileStr)
    
    # Import data
    
    cd("ODE Model/Data/"*datasetStr)
    raw_data = permutedims(readdlm(fileStr, ',', Float64, '\n', header=true)[1])
    
    data_length = size(raw_data)[2]
    # Timepoints, normalized to 0.0 - 10.0
    data_timepoints = round.(collect(LinRange(0.0,10.0, data_length)), digits=2)
    
    data = raw_data[1:end .!= 1, 1:end] # Remove the timepoints from the matrix
    # Result: matrix with ROWS = cell states, COLUMN = timepoints
    
    # Fit to model
    Turing.setadbackend(:forwarddiff)
    data_model = fit_model(data, data_timepoints)
    
    return data_model, data, data_timepoints
end;

In [ ]:
function run_chains(data_model, figSaveFolder, paramSaveFolder, datasetStr, repStr)
    
    # Run chain
    chain = mapreduce(c -> sample(data_model, NUTS(.65), 1250), chainscat, 1:3)
    plot(chain, dpi=250)
    
    # Save chain parameter distribution figures
    cd("ODE Model/"*figSaveFolder)
    savefig(datasetStr*" - "*repStr*", chains result.png")
    
    # Save parameter results from chain iterations
    cd("ODE Model/"*paramSaveFolder)
    CSV.write(datasetStr*" - "*repStr*", param results.csv", DataFrame(chain));
    
    return chain
end;

## ODE Model

In [ ]:
# vanDijk runs

datasetStr = dataset_strings[2]
unfitted_model()

for i in 1:size(vanDijk_conditions)[1]
    repStr = vanDijk_conditions[i]
    fileStr = vanDijk_files[i]
    stdevStr = vanDijk_files_stdev[i]
    
    data_model, data, data_timepoints = create_data_model(datasetStr, fileStr)
    chain = run_chains(data_model, figSaveFolder, paramSaveFolder, datasetStr, repStr)
end

In [ ]:
# Cook runs
# cook_conditions = ["A549-TGFB1"] - to just test one

datasetStr = dataset_strings[3]
unfitted_model()

for i in 1:size(cook_conditions)[1]
# i = 1
    repStr = cook_conditions[i]
    fileStr = cook_files_pseudotime[i]
    stdevStr = cook_files_stdev[i]

    data_model, data, data_timepoints = create_data_model(datasetStr, fileStr)
    chain = run_chains(data_model, figSaveFolder, paramSaveFolder, datasetStr, repStr)
end

In [ ]:
# Panchy data

datasetStr = "6-Panchy"
repStr = "Panchy"
fileStr = repStr*" - Pseudotime - 15 bins, cropped.csv"
stdevStr = repStr*" - Pseudotime - 15 bins, cropped, stdev.csv"

unfitted_model()
data_model, data, data_timepoints = create_data_model(datasetStr, fileStr)
chain = run_chains(data_model, figSaveFolder, paramSaveFolder, datasetStr, repStr)